NAME: __TODO: FULLNAME__

# Machine Learning Practice - Asynchronous
# Homework 03: Classifiers

## Objectives
* Compute class labels from raw data
* Use your imputing/filtering skills to clean up raw data
* Implement a classifier that predicts robot motion from infant movements
* Evaluate the classifier by:
  + Using built-in cross-validation tools
  + Computing TPR/FPRs
  + Displaying their CDFs
  + Displaying the corresponding ROC curve

## Instructions
* All Homework must be individual work.  Do not look at or copy solutions of other students or that are available on the Internet or via LLMs
* Only work in a copy of the file that is from your ~/homework_in/ directory
   + __If you do not use your own copy of this file, then it is an automatic zero on the assignment__
* Read the code below 
* For any cell that is flagged as *TODO*, complete the code according to the specifications
* Execute each cell and verify that it is showing correct results.  Note that because we are reusing variables, the order of execution is *really* important.
* Hand-In Procedure
  + Make sure that your notebook has been saved.  You are responsible for ensuring that the copy that you submit is current and complete
  + The name of the file should be the same as what we gave you
  + Download this file to your local machine (extension: .ipynb)
  + Submit to the Gradescope Notebook HW03 dropbox

## General References
* [Python Built-in Functions](https://docs.python.org/3/library/functions.html)
* [Python Data Structures](https://docs.python.org/3/tutorial/datastructures.html)
* [Numpy Reference](https://docs.scipy.org/doc/numpy/reference/index.html)
* [Summary of matplotlib](https://matplotlib.org/3.1.1/api/pyplot_summary.html)
* [Pandas DataFrames](https://urldefense.proofpoint.com/v2/url?u=https-3A__pandas.pydata.org_pandas-2Ddocs_stable_reference_api_pandas.DataFrame.html&d=DwMD-g&c=qKdtBuuu6dQK9MsRUVJ2DPXW6oayO8fu4TfEHS8sGNk&r=9ngmsG8rSmDSS-O0b_V0gP-nN_33Vr52qbY3KXuDY5k&m=mcOOc8D0knaNNmmnTEo_F_WmT4j6_nUSL_yoPmGlLWQ&s=h7hQjqucR7tZyfZXxnoy3iitIr32YlrqiFyPATkW3lw&e=)
* [Sci-kit Learn Linear Models](https://scikit-learn.org/stable/api/sklearn.linear_model.html)
  + [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)
* [Sci-kit Learn Ensemble Models](https://scikit-learn.org/stable/api/sklearn.ensemble.html)
* [Sci-kit Learn Metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html)
* [Sci-kit Learn Model Selection](https://scikit-learn.org/stable/api/sklearn.model_selection.html)


In [ ]:
import pandas as pd
import numpy as np
import os, re, fnmatch
import matplotlib.pyplot as plt
import matplotlib.patheffects as peffects

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, confusion_matrix, roc_curve, auc
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
import scipy.stats as stats

# Default figure parameters
plt.rcParams['figure.figsize'] = (8,4)
plt.rcParams['font.size'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 16

%matplotlib inline

# LOAD DATA

In [ ]:
""" TODO
Load data from subject k2 for week 5
Display info() for the data

These are data obtained from a baby on the SIPPC. 3D Position (i.e. kinematic)
data are collected at 50 Hz, for the x, y, and z positions in meters, for 
various joints such as the wrists, elbows, shoulders, etc.
"""
# Local file name
fname = '/mlp/datasets/baby1/subject_k2_w05.csv'

baby_data_raw = #TODO
baby_data_raw.info()

In [ ]:
""" TODO
Display the first few examples
"""
# TODO

In [ ]:
""" TODO
Display the last few examples
"""
# TODO

In [ ]:
""" TODO
Display the summary statistics
"""
 # TODO

In [ ]:
""" TODO
Check the dataframe for any NaNs using pandas methods
isna() and any() for a summary of the missing data
"""
# TODO

# Data Selection

In [ ]:
""" PROVIDED
"""
## Support for identifying kinematic variable columns
def get_kinematic_properties(data):
    # Regular expression for finding kinematic fields
    regx = re.compile("_[xyz]$")

    # Find the list of kinematic fields
    fields = list(data)
    fieldsKin = [x for x in fields if regx.search(x)]
    return fieldsKin

def position_fields_to_velocity_fields(fields, prefix='d_'):
    '''
    Given a list of position columns, produce a new list
    of columns that include both position and velocity
    '''
    fields_new = [prefix + x for x in fields]
    return fields + fields_new


In [ ]:
""" PROVIDED
Get the names of the sets of fields for the kinematic features and the 
velocities
"""
fieldsKin = get_kinematic_properties(baby_data_raw)
fieldsKinVel = position_fields_to_velocity_fields(fieldsKin)
print(fieldsKinVel)

"""
Fields that describe the linear and rotational velocities of the robot
"""
fieldsRobot = ['robot_vel_l', 'robot_vel_r']


# Construct Pipeline Components

Copy your pipeline class implementations from HW03

In [ ]:
# DataFrameSelector


In [ ]:
# InterpolationImputer


In [ ]:
# Filter

def computeBoxWeights(length=3):
    '''
    PROVIDED
    
    Computes the kernel weights for a Box Filter 
    
    :param length: the number of terms in the filter kernel
    :return: a vector of the specified length
    '''
    
    return np.ones((length,))/length 



In [ ]:
'''
PROVIDED
'''

class DerivativeComputer(BaseEstimator, TransformerMixin):
    def __init__(self, attribs=None, prefix='d_', dt=1.0):
        self.attribs = attribs
        self.prefix = prefix
        self.dt = dt
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        '''
        :param X: a DataFrame
        :return: a DataFrame with additional features for the derivatives
        '''
        Xout = X.copy()
        if self.attribs is None:
            self.attribs = Xout.columns

        # Iterate over all of the attributes that we need to compute velocity over
        for attrib in self.attribs:
            # Extract the numpy array of data
            vals = Xout[attrib].values
            # Compute the difference between neighboring timeseries elements
            diff = vals[1:] - vals[0:-1]
            # Take into account the amount of time between timeseries samples
            deriv = diff / self.dt
            # Add a zero to the end so the resulting velocity vector is the same
            #   length as the position vector
            deriv = np.append(deriv, 0)
            
            # Add a new derivative attribute to the DataFrame
            attrib_name = self.prefix + attrib
            Xout[attrib_name] = pd.Series(deriv)

        return Xout

# Construct Pipelines

In [ ]:
""" TODO
Create four pipelines. 

The first pipeline is used for the raw dataframe:
1.  Impute values for the kinematic features using a quadratic imputer
2.  Smooth the kinematic features.  Use a Box Filter of length 9 
3.  Compute derivatives of all of the kinematic features.  dt is 0.02 seconds
The output is a cleaned data frame.

The cleaned data frame will be input to several additional pipelines:

The second pipeline extracts the kinematic and velocity (derivative)
features from the dataframe.

The third pipeline extracts the time stamp from the dataframe.

The fourth pipeline extracts the robot velocity from the dataframe (both the linear and rotational velocity).
"""
# Sampling rate: number of seconds between each time sample
dt = .02


# Initial pre-processing
pipe_preprocessor = Pipeline([
 # TODO
])

# Position, velocity selector
pipe_kin_vel = Pipeline([
     #TODO
])

# Time selector
pipe_time = Pipeline([
    #TODO
])

# Robot velocity selector
pipe_robot_vel = Pipeline([
    # TODO
])

In [ ]:
fieldsKinVel

## Pre-process and extract data

In [ ]:
""" TODO
Use the above pipelines to extract the data with kinematic and velocity 
features, the time, and the robot velocity.

See the lecture on classifers for examples.
"""
# TODO: use the first pipeline to perform an initial cleaning of the data
baby_data_clean =  # TODO

# TODO: Use the result from the first pipeline to extract the kinematic and 
#       velocity features by using the pipe_kin_vel pipeline
data_pos_vel = # TODO

# TODO: Use the result from the first pipeline to extract the time stamps by using
#       the pipe_time pipeline
data_time = # TODO


# TODO: Use the result from the first pipeline to get the robot velocity by using
#       the pipe_robot_vel pipeline
data_robot_vel = #TODO

# PROVIDED: Transform the dataframes as numpy arrays
inputs_pos_vel = data_pos_vel.values
time = data_time.values
robot_vel = data_robot_vel.values

nsamples = #TODO

## Examine Robot Velocity

In [ ]:
""" TODO
Create a plot that contains both the linear velocity (robot_vel[:,0]) and
rotational velocity (robot_vel[:,1]).  The plot should contain appropriate 
labels

Note: units are m/s and rad/s, respectively
"""

# TODO
plt.figure()
plt.plot(time, robot_vel)


In [ ]:
""" PROVIDED
Create labels that correspond to "fast backward motion" and
"fast right rotational motion"

"""
# Fast backward motion
labels_linear = robot_vel[:,0] < -0.0025

# Rightward turns
labels_rotational = (robot_vel[:,1]) < -0.02

In [ ]:
""" TODO
Augment the figure you created above to show the two newly-created
class labels.  Make sure that the resulting figure is easy to read
"""
# TODO
plt.figure()


## First Classifier
Create an instance of the SGDClassifier and fit our entire data set using this data.

Details: Random_state=1138, max_iter=10000, tol=1e-3, and
that uses the log_loss function. Fit the model using the position x, y, z
and velocity x, y, z for all limbs as the input features to the model. Use
the robot linear velocity labels as the desired output of the model.

In [ ]:
""" TODO
"""
# Input
X = inputs_pos_vel

# Desired output
y = labels_linear

# TODO: Create and fit the classifer
clf = # TODO
clf.fit(X, y)

# TODO: extract the predictions and the decision function scores from the model for the 
#  entire data set
preds = #TODO

scores = #TODO

In [ ]:
""" PROVIDED
"""
# Generate a color map plot for a confusion matrix
def confusion_mtx_colormap(mtx, xnames, ynames, cbarlabel="", FIGWIDTH=5, FIGHEIGHT=5, FONTSIZE=14):
    ''' 
    Generate a figure that plots a colormap of a matrix
    PARAMS:
        mtx: matrix of values
        xnames: list of x tick names
        ynames: list of the y tick names
        cbarlabel: label for the color bar
    RETURNS:
        fig, ax: the corresponding handles for the figure and axis
    '''
    nxvars = mtx.shape[1]
    nyvars = mtx.shape[0]
    
    # create the figure and plot the correlation matrix
    fig, ax = plt.subplots(figsize=(FIGWIDTH, FIGHEIGHT))
    im = ax.imshow(mtx, cmap='summer')
    if not cbarlabel == "":
        cbar = ax.figure.colorbar(im, ax=ax)
        cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")
    
    # Specify the row and column ticks and labels for the figure
    ax.set_xticks(range(nxvars))
    ax.set_yticks(range(nyvars))
    ax.set_xticklabels(xnames)
    ax.set_yticklabels(ynames)
    ax.set_xlabel("Predicted Labels")
    ax.set_ylabel("Actual Labels")

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, 
             ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    lbl = np.array([['TN', 'FP'], ['FN', 'TP']])
    for i in range(nyvars):
        for j in range(nxvars):
            text = ax.text(j, i, "%s = %.3f" % (lbl[i,j], mtx[i, j]),
                           ha="center", va="center", color="k")
            #text.set_path_effects([peffects.withStroke(linewidth=2, 
            #foreground='w')])

    return fig, ax


def display_confusion_matrix(y, preds, label_names):
    """ 
    Compute the confusion matrix using sklearn's confusion_matrix() function and 
    generate a color map using the provided confusion_mtx_colormap() for the model 
    built using the distance labels.
    
    :params y: Ground truth labels
    :params preds: Crisp predictions made by the model (i.e., after thresholding)
    :return: Number of positive and negative examples (ground truth)
    """
    dist_confusion_mtx = confusion_matrix(y, preds)
    confusion_mtx_colormap(dist_confusion_mtx, label_names, label_names, cbarlabel="") # TODO

    nneg = dist_confusion_mtx[0].sum()
    npos = dist_confusion_mtx[1].sum()
    return npos, nneg

In [ ]:
'''
TODO: Complete the visualization implementations
'''
def visualize_model_output_timeseries(y, preds, scores, threshold=0, offset_pred=-2, offset_scores=-8):
    '''
    Plot timeseries on a single axis:
    1. True class (y)
    2. Predicted class (preds)
    3. Prediction scores (scores)
    
    In addition, draw a horizontal line over the scores that shows the decision threshold 
    (by default the decision threshold is zero)
    
    Don't forget to supply a meaningful legend and to label the horizontal axis
    '''
    
    plt.figure()
    # TODO
    
    plt.xlabel("Time (s)")
    plt.legend()
    
    
'''
TODO

Compute the ROC Curve and generate the KS plot
'''
def ks_roc_plot(targets, scores, FIGWIDTH=12, FIGHEIGHT=6, FONTSIZE=16):
    ''' 
    Generate a figure with two plots:
    1. Distributions of the TPR and FPR over a set of thresholds.  Include
    a vertical line that shows the threshold that maximizes the difference 
    between TPR and FPR
    2. ROC Curve.  Show the point on the curve that corresponds to the same
    threshold
    
    PARAMS:
        targets: list of true target labels
        scores: list of predicted scores
    RETURNS:
        fpr: false positive rate
        tpr: true positive rate
        thresholds: thresholds used for the ROC curve
        auc: Area under the ROC Curve
        fig, axs: corresponding handles for the figure and axis
    '''
    fpr, tpr, thresholds =  # TODO
    diff = tpr - fpr
    auc_res =  # TODO
    elem_max = np.argmax(diff)
    thresh_max = # TODO
    print('K-S Distance:', diff[elem_max])

    # Generate figure with two axes
    fig, ax = plt.subplots(1, 2, figsize=(FIGWIDTH,FIGHEIGHT))
    axs = ax.ravel()  # Individual axes are ax[0] and ax[1]
    
    # TODO: plot TPR, FPR and difference
    
    ax[0].legend(['TPR', 'FPR', 'Difference'], fontsize=FONTSIZE)
    
    # TODO: Generate ROC Curve plot
    
    auc_text = ax[1].text(.05, .95, "AUC = %.4f" % auc_res, 
                          color="k", fontsize=FONTSIZE)
    print("AUC:", auc_res)

    return fpr, tpr, thresholds, thresh_max, auc_res, fig, axs


""" 
TODO

Plot histograms of the scores from the model.
1. Histogram of all scores
2. Overlapping histograms of the scores for the positive and negative examples

Make sure to include a horizontal line at the best threshold (K-S threshold).
"""

def plot_score_histograms(scores, y, best_thresh=None, nbins = 41, FIGWIDTH=8, FIGHEIGHT=4):
    '''
    Generate two plots:
    1. Histogram of all scores
    2. Two histograms: one for positive examples and the other for negative examples
    
    :param scores: Model scores for all samples
    :param y: Ground truth labels for all samples
    '''

    scores_pos = [s for (s, l) in zip(scores, y) if l]     # TODO
    scores_neg = [s for (s, l) in zip(scores, y) if not l] # TODO

    plt.figure(figsize=(FIGWIDTH, FIGHEIGHT))
    plt.subplot(1,2,1)
    # TODO
    n, bins, patches = plt.hist(scores, bins=nbins)
    if best_thresh is not None:
        # TODO plot vertical line
        
    plt.xlabel('score')
    plt.ylabel('count')

    plt.subplot(1,2,2)
    # TODO
   
        
    plt.xlabel('score')
    plt.legend(loc='upper right')

In [ ]:
# EXECUTE CELL: Visualize the predictions made by the model in timeseries form
visualize_model_output_timeseries(y, preds, scores)

In [ ]:
# EXECUTE CELL
display_confusion_matrix(y, preds, ['not', 'fast backward'])


In [ ]:
# EXECUTE CELL: Generate the TPR/FPR and ROC plots
fpr, tpr, thresholds, best_thresh, auc_res, fig, axs = ks_roc_plot(y, scores) 

In [ ]:
# EXECUTE CELL: Plot score histograms
plot_score_histograms(scores, y, best_thresh)

## Rotational Velocity Classifier

Create a new classifier that predicts the 'fast rightward velocity' label.  Use the same
parameters for the classifier as above.

In [ ]:
""" TODO
"""
# Input
X = inputs_pos_vel

# Desired output
y = labels_rotational

# TODO: Create and fit the classifer
clf =  # TODO


# TODO: extract the predictions and the decision function scores from the model for the entire data set
preds = # TODO

scores = # TODO

In [ ]:
# EXECUTE CELL: Visualize the predictions made by the model in timeseries form
visualize_model_output_timeseries(y, preds, scores)

In [ ]:
# EXECUTE CELL
display_confusion_matrix(y, preds, ['not', 'fast right'])


In [ ]:
# EXECUTE CELL: Generate the TPR/FPR and ROC plots
fpr, tpr, thresholds, best_thresh, auc_res, fig, axs = ks_roc_plot(y, scores) 

In [ ]:
# EXECUTE CELL: Plot score histograms
plot_score_histograms(scores, y, best_thresh)

# Reflection
Please provide short answers to each of the questions

_Q1. Examining the two ground truth labels, to what degree do they overlap in time?  (i.e., how often is the robot backing up at the same time that it is turning to the right)?_

**TODO**


_Q2. Which of the two models exhibit the highest Kolmogorov-Schmirnov Distance?_

**TODO**


_Q3. Did your pre-processing stage eliminate all of the NaNs in your data?  How do you know?_

**TODO**

_Q4.  Which model exibits the best AUC?_

**TODO**




# Classification Using Cross-Validation

So far, we have used the same data set for both training the model and testing it.  This can give us a very skewed picture of the model's capability to perform well with unused data.  Here, we will use simple Cross-Validation to simulate model performance on unseen data.

In [ ]:
""" TODO
LINEAR VELOCITY

Create another SGDClassifier with the same parameters to predict the linear
velocity label as a function of the kinematic positions and velocities.

W will use cross_val_predict() to fit N models, and compute 
predictions for each sample and their corresponding scores. Use 30 cross 
validation splits (i.e. cv=30).

"""
# Model input
X = inputs_pos_vel
# Model output
y = labels_linear

# TODO: Create and fit the classifer
clf3 =  # TODO

# TODO: use cross_val_predict() to compute the scores by setting the 'method'
#       parameter equal to 'decision_function'. Please see the reference 
#       links above
scores = # TODO

# TODO: use cross_val_predict() to compute the predicted labels by setting 
#       the 'method' parameter equal to 'predict'. Please see the reference 
#       links above
preds = # TODO

In [ ]:
# EXECUTE CELL: Visualize the predictions made by the model in timeseries form
visualize_model_output_timeseries(y, preds, scores)

In [ ]:
# EXECUTE CELL
display_confusion_matrix(y, preds, ['not', 'fast backward'])


In [ ]:
# EXECUTE CELL: Generate the TPR/FPR and ROC plots
fpr, tpr, thresholds, best_thresh, auc_res, fig, axs = ks_roc_plot(y, scores) 
x = np.argmax(tpr-fpr)
print("Best:", x, best_thresh)

In [ ]:
# Plot score histograms
plot_score_histograms(scores, y, best_thresh)

## Cross-validation for Rotational Velocity

In [ ]:
""" TODO
ROTATIONAL VELOCITY

Take the same cross-validation approach for the rotational velocity label

""" 
# Model input
X = inputs_pos_vel
# Model output
y = labels_rotational

# TODO: Create and fit the classifer
clf4 = # TODO
#clf.fit(X, y)

# TODO: use cross_val_predict() to compute the scores by setting the 'method'
#       parameter equal to 'decision_function'. Please see the reference 
#       links above
scores = # TODO
    
# TODO: use cross_val_predict() to compute the predicted labels by setting 
#       the 'method' parameter equal to 'predict'. Please see the reference 
#       links above (unfortunately, we have to refit the models)
preds = # TODO

In [ ]:
# EXECUTE CELL: Visualize the predictions made by the model in timeseries form
visualize_model_output_timeseries(y, preds, scores)

In [ ]:
# EXECUTE CELL
display_confusion_matrix(y, preds, ['not', 'fast right'])


In [ ]:
# EXECUTE CELL: Generate the TPR/FPR and ROC plots
fpr, tpr, thresholds, best_thresh, auc_res, fig, axs = ks_roc_plot(y, scores) 
x = np.argmax(tpr-fpr)
print("Best:", x, best_thresh)

In [ ]:
# Plot score histograms
plot_score_histograms(scores, y, best_thresh)

# Reflection, Part 2

Write a short answer to each of the following questions:

_Q5. Looking at the results from the first and second linear velocity models:  What is the difference in their performance and how do you explain this difference?_

**TODO**

_Q6. Looking at the results from the first and second rotational velocity models: which performance results are best to report when discussing model performance?_

**TODO**


_Q7.  Looking at the linear velocity cross-validation TPR/FPR curves: what is the ideal threshold to use that distinquishes between the two classes?  Which threshold is used for the corresponding confusion matrix plot?_

**TODO**


_Q8. What is the False Positive Rate for the rotational velocity cross-validated model at the threshold that maximizes the difference between TPR and FPR?_

**TODO**



